<a href="https://colab.research.google.com/github/uppercaveman/vllm-learn/blob/main/vllm_llama3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 检查Python环境

In [ ]:
!pip --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [ ]:
!pip3 --version

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


# 查看GPU信息

In [ ]:
# 选择并连接T4服务器
# nvidia-smi 是 NVIDIA 提供的一个命令行工具，用于监控和管理 NVIDIA GPU 设备
!nvidia-smi

Mon Apr 22 07:25:59 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# 更多内存信息
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install -q gputil

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    virtual_memory = humanize.naturalsize( psutil.virtual_memory().available )
    memory_info = humanize.naturalsize( process.memory_info().rss)
    memoryFree =gpu.memoryFree
    memoryUsed = gpu.memoryUsed
    memoryUtil = gpu.memoryUtil*100
    memoryTotal = gpu.memoryTotal

    print(f"Gen RAM Free: {virtual_memory} | Proc size: {memory_info}" )

    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".\
          format(memoryFree, memoryUsed, memoryUtil*100, memoryTotal))
printm()

  Preparing metadata (setup.py) ... done
Gen RAM Free: 12.6 GB | Proc size: 98.5 MB
GPU RAM Free: 15101MB | Used: 0MB | Util   0% | Total 15360MB


In [ ]:
# 查看cuda版本信息
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


# 安装vllm

In [ ]:
# 源码安装，下载代码
!git clone https://github.com/vllm-project/vllm.git vllmsrc

Cloning into 'vllm'...
remote: Enumerating objects: 15441, done.
remote: Counting objects: 100% (4577/4577), done.
remote: Compressing objects: 100% (735/735), done.
remote: Total 15441 (delta 4140), reused 3881 (delta 3842), pack-reused 10864
Receiving objects: 100% (15441/15441), 7.85 MiB | 21.95 MiB/s, done.
Resolving deltas: 100% (11627/11627), done.


In [ ]:
# 安装vllm
!cd vllmsrc && pip3 install -e .

Obtaining file:///content/vllmsrc
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for vllm (pyproject.toml) ... done
  Created wheel for vllm: filename=vllm-0.4.1+cu122-0.editable-cp310-cp310-linux_x86_64.whl size=10361 sha256=573534eb7bdacf8ca0d198d85c594434577c6a0050898b2386e7056010090194
  Stored in directory: /tmp/pip-ephem-wheel-cache-ypnxw7w_/wheels/40/72/a8/b1baf9a42a4dfad711a1789b685cfb16d9bc26f04ea54ffd0f
Successfully built vllm
  Attempting uninstall: vllm
    Found existing installation: vllm 0.4.1+cu122
    Uninstalling vllm-0.4.1+cu122:
      Successfully uninstalled vllm-0.4.1+cu122


In [ ]:
# 检查vllm安装情况
!pip3 show vllm

Name: vllm
Version: 0.4.1+cu122
Summary: A high-throughput and memory-efficient inference and serving engine for LLMs
Home-page: https://github.com/vllm-project/vllm
Author: vLLM Team
Author-email: 
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Editable project location: /content/vllm
Requires: cmake, fastapi, filelock, lm-format-enforcer, ninja, numpy, outlines, prometheus-client, psutil, py-cpuinfo, pydantic, pynvml, ray, requests, sentencepiece, tiktoken, tokenizers, torch, transformers, typing-extensions, uvicorn, vllm-nccl-cu12, xformers
Required-by: 


In [ ]:
# 检查torch版本
import torch
print(torch.version.cuda)

12.1


# 测试

In [ ]:
import vllm
from vllm import LLM, SamplingParams

ImportError: cannot import name 'LLM' from 'vllm' (unknown location)

In [ ]:
# 定义输入提示列表和生成的采样参数
prompts = [
    "你好，我的名字是",
    "中国首都是",
    "北京水库是",
    "AI的未来是",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

ImportError: cannot import name 'LLM' from 'vllm' (unknown location)

In [ ]:
llm = LLM(model="meta-llama/Meta-Llama-3-8B-Instruct")

# 排查 Import "vllm" could not be resolved 问题

In [ ]:
!pip install vllm

In [ ]:
!echo $PYTHONPATH

/env/python


In [ ]:
# 设置环境变量
# %env PYTHONPATH=/env/python:/usr/local/lib/python3.10/dist-packages
%env PYTHONPATH=/env/python
!echo $PYTHONPATH

env: PYTHONPATH=/env/python
/env/python


In [ ]:
!whereis python

python: /usr/local/bin/python


In [ ]:
!python --version

Python 3.10.12


In [ ]:
!whereis pip

pip: /usr/local/bin/pip


In [ ]:
!pip --version

In [ ]:
!mv vllm vllmsrc